In [ ]:
import cvxpy as cp
import numpy as np
from numpy.linalg import norm

# Placeholder for loading data
# Demand, DayC, Time, Past are arrays that you need to load or define based on your actual data
Demand = np.random.rand(1000)  # Example placeholder
DayC = np.random.rand(1000, 1)  # Example placeholder for day context feature
Time = np.random.rand(1000, 1)  # Example placeholder for time feature
Past = np.random.rand(1000, 24)  # Example placeholder for past observations feature

lntr = 12*7*16  # Training data size
lnva = lntr // 2  # Validation data size
pvec = [12 * i for i in range(10, 25, 2)]  # Example periods to consider

b = 2.5 / 3.5
h = 1 / 3.5
r = b / (b + h)

for p in pvec:
    if p == 0:
        Features = np.hstack([DayC, Time])
    else:
        Features = np.hstack([DayC, Time, Past[:, :p]])
    lF = Features.shape[1]
    
    # Initializing storage for results
    Qfac = np.zeros((lnva, 1 + lF))
    QfacD = np.zeros(lnva)
    muD = np.zeros(lnva)
    sigmaD = np.zeros(lnva)
    
    for t in range(lntr + 1, lntr + lnva + 1):
        # Normalize features
        FeaturesT = Features[t - lntr:t, :] / norm(Features[t - lntr:t, :], np.inf, axis=0)
        
        # OLS to estimate coefficients
        beta0 = cp.Variable()
        beta1 = cp.Variable(lF)
        Res = cp.Variable(lntr)
        
        objective = cp.Minimize(cp.sum_squares(Res))
        constraints = [Res == Demand[t - lntr + i + delay - 1] - beta0 - beta1.T @ FeaturesT[i, :] for i in range(lntr)]
        prob = cp.Problem(objective, constraints)
        prob.solve()
        
        # Estimating sigma via OLS (not directly translatable; needs adjustment)
        # In Python, you would typically use statistical packages for such estimation
        
        # Optimal order quantity calculation (adjust formula as needed)
        current_features = Features[t, :] / norm(Features[t - lntr:t, :], np.inf)
        muD[t - lntr] = beta0.value + beta1.value.T @ current_features
        sigmaD[t - lntr] = np.exp(0)  # Placeholder for actual sigma calculation
        
        # Adjust the following calculation as per your actual minimax approach
        QfacD[t - lntr] = muD[t - lntr] + sigmaD[t - lntr] * np.sqrt(r)  # Example adjustment

# Save results; in Python, use numpy.save, scipy.io.savemat, or similar
